# Objective:  Analyze an internal email network of employees and determine which nodes need to be removed to disrupt information flow

I will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx

In [2]:
def plot_graph(G, weight_name=None):   
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Step 1

Using networkx, load up the directed multigraph from `email_network.txt`. The node names are strings.

*This function returns a directed multigraph networkx graph.*

In [3]:
def step_one():
    
    G = nx.read_edgelist("email_network.txt", delimiter = '\t', data = [('timestamp', int)], create_using=nx.MultiDiGraph())
    
    return G

### Step 2

Determine the number employees and emails represented in the graph from Step 1

*This function returns a tuple (#employees, #emails).*

In [4]:
def step_two():
        
    G = step_one()
    num_employees = len(G.nodes())
    num_emails = len(G.edges())
    answer = (num_employees,num_emails)
    
    return answer

In [5]:
step_two()

(167, 82927)

### Step 3

Part a: Assuming that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data I determine if it possible for information to go from every employee to every other employee.


Part b: Assuming that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data I determine if it is possible for information to go from every employee to every other employee.


*This function returns an answer in the form of a tuple of bools (parta, partb).*

In [6]:
def step_three():
    #a network that is strongly connected would mean that it is possible for information to go 
    #from every employee "node" to every other employee "node"
    G = step_one()
    pt_a_answer= nx.is_strongly_connected(G)

    # a network is weakly connected if when the directional edges are turned into bidirectional edges
    # all employee "nodes" are still connected to each other
    # convert G to an undirected network
    G_undirected = G.to_undirected()
    pt_b_answer=nx.is_connected(G_undirected)

    answer = (pt_a_answer,pt_b_answer)   
    return answer

In [7]:
step_three()

(False, True)

### Step 4

I determine how many nodes are in the largest (in terms of nodes) weakly connected component.

*This function returns an int.*

In [8]:
def step_four():    
    G = step_one()

    #break the network into its weakly_connected_components
    wccs = nx.weakly_connected_components(G)

    #return the wccs whose len is the largest
    largest_wccs= max(wccs, key = len)

    #get the size of the largest_wccs
    answer = len(largest_wccs)
    
    return answer

In [9]:
step_four()

167

### Step 5

I determine the number of nodes in the largest (in terms of nodes) strongly connected component.

*This function returns an int*

In [10]:
def step_five():
    G = step_one()
    #seperate the network into it's strongly_connected_components(G)
    sccs = nx.strongly_connected_components(G)
    largest_sccs = max(sccs, key = len)
    answer = len(largest_sccs)
    return answer

In [11]:
step_five()

126

### Step 6

Using the NetworkX function strongly_connected_component_subgraphs, I find the subgraph of nodes in a largest strongly connected component. 
I call this graph G_sc.

*This function returns a networkx MultiDiGraph named G_sc.*

In [12]:
def step_six():
        
    G = step_one()
    #generate the strongly connected component as subgraphs
    scc_subs = nx.strongly_connected_component_subgraphs(G)

    #get the largest of the strongly connected component subgraphs
    G_sc = max(scc_subs, key = len)

    answer = G_sc
    
    return answer

In [13]:
step_six()

### Step 7

I determine the average distance between nodes in G_sc 

*This function returns a float.*

In [14]:
def step_seven():
        
    G = step_six()
    answer = nx.average_shortest_path_length(G)
    
    return answer

In [15]:
step_seven()

1.6461587301587302

### Step 8

I determine the largest possible distance between two employees in G_sc 

*This function returns an int.*

In [16]:
def step_eight():
        
    G = step_six()

    #the largest possible distance is the diameter of the network
    answer = nx.diameter(G)
    
    return answer

In [17]:
step_eight()

3

### Step 9

I determine the set of nodes in G_sc with eccentricity equal to the diameter 

*This function returns a set of the node(s).*

In [18]:
def step_nine():
       
    G = step_six()

    #nodes's whose eccentricity is equal to the diameter is the same as finding the periphery
    answer = set(nx.periphery(G))
    
    return answer

In [19]:
step_nine()

{'129', '134', '97'}

### Step 10

I determine the set of node(s) in G_sc with eccentricity equal to the radius 

*This function returns a set of the node(s).*

In [20]:
def step_ten():
        
    # nodes whose eccentricity is equal to the radius is the same as finding the center
    G = step_six()
    answer = set(nx.center(G))
    
    return answer

In [21]:
step_ten()

{'38'}

### Step 11

I determine which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc 

I determine how many nodes are connected to this node 


*This function returns a tuple (name of node, number of satisfied connected nodes).*

In [22]:
def step_eleven():
        
    G = step_six()

    #get the diameter of G_sc
    d = nx.diameter(G)

    #nodes whose shortest path of length equals the diameter are the peripheries
    peripheries = nx.periphery(G)

    #initialize variables
    max_count = -1
    result_node = None

    #go through each node in the periphery
    for node in peripheries:
        #calculate the shortest path lengths between the periphery node and all other nodes in the 
        #network
        sp = nx.shortest_path_length(G, node)
        #count how many times the shortest length path is the same as the diameter
        #this counts how often a node is connected to other nodes whose distance between nodes
        #is the same as the diameter
        count = list(sp.values()).count(d)
        #if the node in Step has more connections than the prior max_count, store this node
        if count > max_count:
            result_node = node
            max_count = count

    answer = (result_node, max_count)
    
    return answer

In [23]:
step_eleven()

('97', 63)

### Step 12

Suppose I want prevent communication from flowing to the node that I found in the previous Step from any node in the center of G_sc. I determine the smallest number of nodes I would need to remove from the graph

*This function return an integer.*

In [24]:
def step_twelve():
        
    G = step_six()

    #calculate the center node
    center = nx.center(G)[0]

    #get the node from the previous Step
    node = step_eleven()[0]

    #return the number minimum set of nodes that disconnect between the center node and the target node
    #this would prevent removing center nodes and previous Steps
    answer = len(nx.minimum_node_cut(G, center, node))
    
    return answer

In [25]:
step_twelve()

0

### Step 13

I construct an undirected graph G_un using G_sc (I ignore the attributes).

*This function returns a networkx Graph.*

In [26]:
def step_thirteen():
    G = step_six()

    #construct and undirected graph
    undir_subgraph = G.to_undirected()

    #convert the undirected graph into a networkx graph
    G_un = nx.Graph(undir_subgraph)

    answer = G_un
    
    return answer

In [27]:
step_thirteen()

### Step 14

I determine the transitivity and average clustering coefficient of graph G_un 

*This function returns a tuple (transitivity, avg clustering).*

In [28]:
def step_fourteen():
        
    G = step_thirteen()

    #get the transitivity
    transitivity = nx.transitivity(G)

    #get the average clustering coefficient
    average_clustering = nx.average_clustering(G)

    answer = (transitivity, average_clustering)
    
    return answer

In [29]:
step_fourteen()

(0.570111160700385, 0.697527243723142)